In [1]:
from __future__ import print_function

from keras import layers
import keras
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D

import numpy as np
import random

Using TensorFlow backend.


In [2]:
from keras.models import load_model
from __future__ import print_function

from keras import backend as K
from keras.engine import Input, Model
import numpy as np
import random
K.set_learning_phase(0)
model = load_model('sample_cnn_model.pkl')

In [3]:
from sklearn.externals import joblib
partition = joblib.load('partition_sample_cnn.pkl')

In [12]:
len(partition['train'])

1115

In [4]:
np.random.seed(3)
label_idx = {'feces': 0 , 'tongue': 1, 'skin': 2}
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, label_idx, batch_size=32, dim=(5000,125,4),
                 n_classes=3, shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = label_idx
        self.list_IDs = list_IDs
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim)
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size) , dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            x_tmp = joblib.load(ID)
            label_tmp = ID.split('_')[1].split('.')[0]
            idx = np.random.randint(x_tmp.shape[0], size=5000)
            X[i,] = x_tmp[idx,:,:]

            # Store class
            y[i] = self.labels[label_tmp]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [5]:
params = {'batch_size': 64,
          'n_classes': 3,
          'shuffle': True}

# Datasets

# Generators
training_generator = DataGenerator(partition['train'], label_idx, **params)
testing_generator = DataGenerator(partition['test'], label_idx, **params)

In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 5000, 125, 4)      0         
_________________________________________________________________
conv0 (Conv2D)               (None, 5000, 120, 4)      100       
_________________________________________________________________
bn0 (BatchNormalization)     (None, 5000, 120, 4)      20000     
_________________________________________________________________
activation_1 (Activation)    (None, 5000, 120, 4)      0         
_________________________________________________________________
max_pool_0 (MaxPooling2D)    (None, 5000, 30, 4)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 5000, 20, 16)      720       
_________________________________________________________________
bn1 (BatchNormalization)     (None, 5000, 20, 16)      20000     
__________

In [7]:
acc = model.evaluate_generator(generator=testing_generator)
print(acc)

[0.51243501901626587, 0.828125]


In [8]:
acc = model.evaluate_generator(generator=training_generator)
print(acc)

[0.34904084749081554, 0.86672794117647056]
